# Feature Engineering and KNN
Author: Karina Condeixa
__________________________________________

In [5]:
# import packages
import pandas as pd
# from faker import Faker, providers
# from faker.providers.address.de_DE import Provider as DeDeAddressProvider
# from faker.generator import random
# from faker.providers import BaseProvider
# import random

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
# %matplotlib notebook
# %matplotlib 

# import folium
# from geopy.geocoders import Nominatim

In [6]:
item_df_original = pd.read_csv('data/item_df.csv')
# item_df_original = item_df_original[:][1:].squeeze()
item_df_original.head()

,item_category,item_condition,item_postcode,item_timer_days,n_views,n_likes,item_id
0,plants,3,12619,11,101,4,1
1,plants,2,10249,10,90,13,2
2,plants,2,13589,28,9,1,3
3,furniture,3,13159,17,116,8,4
4,kids,3,10318,5,161,20,5


In [7]:
item_df = item_df_original.copy()

## Feature engineering


### Since the dataset is fake, there is no nulls on it.

### One-hot Encoding 

Function to convert categorical variations to numerical ones. The results 0 indicates non-existent while 1 indicates existent.

In [8]:
# columnsToEncode= ['item_category', 'item_condition']

# item_onehot = pd.get_dummies(item_df,columns=columnsToEncode, drop_first=True)
# item_onehot

In [9]:

item_cat_onehot = pd.get_dummies(item_df[['item_category']], prefix="cat", prefix_sep="_")
# item_cat_onehot.head()

# add neighborhood column back to dataframe
item_cat_onehot['item_postcode'] = item_df['item_postcode'] 

# move neighborhood column to the first column
fixed_columns = [item_cat_onehot.columns[-1]] + list(item_cat_onehot.columns[:-1])
item_cat_onehot = item_cat_onehot[fixed_columns]


item_cat_grouped = item_cat_onehot.groupby('item_postcode').mean().reset_index()

# item_onehot.head()

item_cat_grouped.head()

,item_postcode,cat_all,cat_clothes,cat_furniture,cat_kids,cat_plants
0,10119,0.125000,0.000000,0.125000,0.500000,0.250000
1,10178,0.428571,0.142857,0.142857,0.142857,0.142857
2,10179,0.428571,0.285714,0.142857,0.142857,0.000000
3,10243,0.125000,0.250000,0.000000,0.125000,0.500000
4,10245,0.000000,0.000000,0.000000,0.600000,0.400000


In [10]:

# item_onehot = pd.get_dummies(item_df[['item_category']], prefix="cat", prefix_sep="_")
# item_onehot 

# # add neighborhood column back to dataframe
# item_onehot['item_postcode'] = item_df['item_postcode'] 

# # move neighborhood column to the first column
# fixed_columns = [item_onehot.columns[-1]] + list(item_onehot.columns[:-1])
# item_onehot = item_onehot[fixed_columns]

# item_grouped = item_onehot.groupby('item_postcode').mean().reset_index()

# # item_onehot.head()

# item_grouped.head()

In [11]:
item_df['item_condition'] = item_df['item_condition'].astype(str)

In [13]:
item_cond_onehot = pd.get_dummies(item_df[['item_condition']], prefix="condition", prefix_sep="_")
item_cond_onehot 

# add postcode column back to dataframe
item_cond_onehot['item_postcode'] = item_df['item_postcode'] 

# move neighborhood column to the first column
fixed_columns = [item_cond_onehot.columns[-1]] + list(item_cond_onehot.columns[:-1])
item_cond_onehot = item_cond_onehot[fixed_columns]

item_cond_grouped = item_cond_onehot.groupby('item_postcode').mean().reset_index()

item_cond_grouped.head()

,item_postcode,condition_1,condition_2,condition_3
0,10119,0.250000,0.375000,0.375000
1,10178,0.285714,0.428571,0.285714
2,10179,0.142857,0.285714,0.571429
3,10243,0.375000,0.125000,0.500000
4,10245,0.600000,0.000000,0.400000


#### Sorting

In [14]:

def return_most_common_items(row, num_top_items):
    row_items = row.iloc[1:]
    row_items_sorted = row_items.sort_values(ascending=False)

    return row_items_sorted.index.values[0:num_top_items]



### Top items by postcode

In [20]:
num_top_items = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['item_postcode']

for ind in np.arange(num_top_items):
    try:
        columns.append('{}{} Most Common Items'.format(ind+1, indicators[ind]))  
    except:
        columns.append('{}th Most Common Items'.format(ind+1))  

# create a new dataframe
postcodes_cat_sorted = pd.DataFrame(columns=columns)
postcodes_cat_sorted['item_postcode'] = item_cat_grouped['item_postcode']


for ind in np.arange(item_cat_grouped.shape[0]):
    postcodes_cat_sorted.iloc[ind, 1:] = return_most_common_items(item_cat_grouped.iloc[ind, :], num_top_items)

In [16]:
postcodes_cat_sorted.head()

,item_postcode,1st Most Common Items,2nd Most Common Items,3rd Most Common Items,4th Most Common Items,5th Most Common Items
0,10119,cat_kids,cat_plants,cat_all,cat_furniture,cat_clothes
1,10178,cat_all,cat_clothes,cat_furniture,cat_kids,cat_plants
2,10179,cat_all,cat_clothes,cat_furniture,cat_kids,cat_plants
3,10243,cat_plants,cat_clothes,cat_all,cat_kids,cat_furniture
4,10245,cat_kids,cat_plants,cat_all,cat_clothes,cat_furniture


In [23]:
num_top_items = 3

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['item_postcode']

for ind in np.arange(num_top_items):
    try:
        columns.append('{}{} Most Common condition'.format(ind+1, indicators[ind]))  
    except:
        columns.append('{}th Most Common Condition'.format(ind+1))  

# create a new dataframe
postcodes_cond_sorted = pd.DataFrame(columns=columns)
postcodes_cond_sorted['item_postcode'] = item_cond_grouped['item_postcode']


# for ind in np.arange(item_grouped.shape[0]):
# #     print(postcodes_items_sorted.iloc[ind, 1:]) 
# #     print(return_most_common_items(item_grouped.iloc[ind, :], num_top_venues))
#     postcodes_items_sorted.iloc[ind, 1:] = return_most_common_items(item_grouped.iloc[ind, :], num_top_items)

for ind in np.arange(item_cond_grouped.shape[0]):
    postcodes_cond_sorted.iloc[ind, 1:] = return_most_common_items(item_cond_grouped.iloc[ind, :], num_top_items)

In [24]:
postcodes_cond_sorted.head()

,item_postcode,1st Most Common condition,2nd Most Common condition,3rd Most Common condition
0,10119,condition_2,condition_3,condition_1
1,10178,condition_2,condition_1,condition_3
2,10179,condition_3,condition_2,condition_1
3,10243,condition_3,condition_1,condition_2
4,10245,condition_1,condition_3,condition_2


### Cluster postcodes

k-means is used to cluster the postcodes into 5 clusters.

In [25]:
# # set number of clusters
# kclusters = 5

# berlin_grouped_clustering = berlin_grouped.drop('neighborhood', 1)

# # run k-means clustering
# kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(berlin_grouped_clustering)

# # check cluster labels generated for each row in the dataframe
# kmeans.labels_[0:10] 

### Cosine similarity
to calculate user-user and item-item similarities

In [ ]:
# from sklearn.metrics.pairwise import cosine_similarity
# import numpy as np

# # Example vectors
# A = np.array([1, 2, 3])
# B = np.array([4, 5, 6])

# # Reshape the vectors into 2D arrays
# A = A.reshape(1, -1)
# B = B.reshape(1, -1)

# # Calculate cosine similarity
# cosine_similarity = cosine_similarity(A, B)

# print(cosine_similarity)


In [ ]:
## CONTINUE FROM HERE: factorization matrix, encoding

### To the Model

In [ ]:
# model_mock_df = pd.DataFrame(create_model_data(1500)).transpose()
# # model_mock_df.head(5)
# model_mock_df.shape

In [ ]:
# model_mock_df.info()

### Creating csv files

In [ ]:
# user_mock_df.to_csv('data/user_mock_data.csv', index=False)

In [ ]:
# item_mock_df.to_csv('data/item_mock_data.csv', index=False)

In [ ]:
# model_mock_df.to_csv('data/model_mock_data.csv', index=False)

## References:
- [Generate custom datasets using Python Faker](https://blogs.sap.com/2021/05/26/generate-custom-datasets-using-python-faker/)
- [folium_polygon_rectangle_layers](https://www.youtube.com/watch?v=9E9FTJrOJ1E&t=752s)
- [Faker](https://github.com/joke2k/faker/issues/1183)
- [Generating Mock Data with Python! (NumPy, Pandas, & Datetime Libraries)](https://www.youtube.com/watch?v=VJBY2eVtf7o)